# American and Bermudan Option pricer

In [1]:
# Packages used
import numpy as np
import matplotlib.pyplot as plt
from scipy import odr
from helper import GBM_Euler, value_option_schwarz, value_option_schwarz_test, value_option_bermudan

## Simulations Run with Data from paper

American option should be priced at .1144

In [2]:
# Parameters
K=1.1; M=4; r=.06; realizations=8

# Matrix
path_matrix = np.array([
[1.00, 1.09, 1.08, 1.34],
[1.00, 1.16, 1.26, 1.54],
[1.00, 1.22, 1.07, 1.03],
[1.00, 0.93, 0.97, 0.92],
[1.00, 1.11, 1.56, 1.52],
[1.00, .76, 0.77, 0.90],
[1.00, 0.92, 0.84, 1.01],
[1.00, 0.88, 1.22, 1.34]])

In [3]:
# Generate cash flows under Longstaff-Schwarz
cash_flows = value_option_schwarz(M,K,path_matrix, r, realizations, option="put")

# Discount cash flows
for time in range(cash_flows.shape[1]):
    cash_flows[:,time]*=np.exp(-r*time)
print(f'Price of American Option is: {np.sum(cash_flows[0:])/realizations}')

Price of American Option is: 0.11665163113684986


In [4]:
cash_flows_test = value_option_schwarz_test(M,K,path_matrix, r, realizations, option="put")

# Discount cash flows
for time in range(cash_flows.shape[1]):
    cash_flows_test[:,time]*=np.exp(-r*time)
print(f'Price of American Option is: {np.sum(cash_flows_test[0:])/realizations}')    

Price of American Option is: 0.11443433004505696


In [5]:
# Unit test Bermudian pricer by giving it same amount of exericse options as American
exercise_dates = [1,2,3,4]
cash_flow_bermudan = value_option_bermudan(M, K, path_matrix, r, realizations, exercise_dates, option="put")

for time in range(cash_flow_bermudan.shape[1]):
    cash_flow_bermudan[:,time] *= np.exp(-r*time)
print(f'Price of Bermudan Option is: {np.sum(cash_flow_bermudan[0:])/realizations}')    

Price of Bermudan Option is: 0.11561153571203728


In [6]:
# Fewer exercise points should lead to lower option price
exercise_dates = [2,3]
cash_flow_bermudan = value_option_bermudan(M, K, path_matrix, r, realizations, exercise_dates, option="put")

for time in range(cash_flow_bermudan.shape[1]):
    cash_flow_bermudan[:,time] *= np.exp(-r*time)
print(f'Price of Bermudan Option is: {np.sum(cash_flow_bermudan[0:])/realizations}')    

Price of Bermudan Option is: 0.056380739270260896


## Simulation based on GBM

In [112]:
# Variables used
T = 1
K = 95
S = 100
M = 100
sigma = 0.2
r = 0.06/M
realizations = 100000
exercise_dates = np.array([2,4,6,8])

In [113]:
# Generate stock scenarios
s_all = np.array([np.array(GBM_Euler(T, S, sigma, r, M)) for x in range(realizations)])

In [114]:
# Generate path matrix
path_matrix = np.zeros((realizations, M))
for realization in range(realizations):
    path_matrix[realization,:] = s_all[realization]

In [115]:
option_cash_flow = value_option_schwarz(M,K,path_matrix, r, realizations,option="call")

for time in range(option_cash_flow.shape[1]):
    option_cash_flow[:,time]*=np.exp(-r*time)
    
print(np.sum(option_cash_flow[1:])/realizations)

5.091211371341537


In [116]:
option_cash_flow = value_option_schwarz(M,K,path_matrix, r, realizations,option="call")


In [117]:
def option_pricer(option_cash_flow, realizations, M, r):
    '''
    Take option cash flow matrix, discount time step and calculate the average and standard devation
    Input: cash flow matrix, realisations, time points, risk free interest rate
    Output: Option price +/- std
    '''
    
    # Discount 
    for time in range(option_cash_flow.shape[1]):
        option_cash_flow[:,time] *= np.exp(-r*time)
    
    # Subtract values which are not zero and create vector to compute first and second moment
    final_price = np.zeros(realizations)
    
    for i in range(realizations):
        for j in range(M):
            if option_cash_flow[i][j] != 0:
                final_price[i] = option_cash_flow[i][j]
    
    return [np.mean(final_price), np.std(final_price)]

In [118]:
option_pricer(option_cash_flow, realizations, M, r)

[5.091237690670511, 2.1279856167843723]

In [119]:
cash_flows_bermudan = value_option_bermudan(M, K, path_matrix, r, realizations, exercise_dates, option = "call")


for time in range(cash_flows_bermudan.shape[1]):
    cash_flows_bermudan[:,time]*=np.exp(-r*time)
    
print(np.sum(cash_flows_bermudan[1:])/realizations)

9.803907921379512


In [120]:
cash_flows_bermudan = value_option_bermudan(M, K, path_matrix, r, realizations, exercise_dates, option = "call")


In [121]:
option_pricer(cash_flows_bermudan, realizations, M, r)

[9.80400561548188, 13.240573527859242]